<span type="title">Lambda Functions and Stream API</span> | <span type="update">2018-10-22</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Java 8 的新特性：类似于函数式编程的 Lambda API 和 基于 Lambda 的 Stream 流处理 API。此外，也会涉及 Optional 接口和并发编程等方面。</p></span>

# Lambda Functions

函数式编程允许将函数作为入参传递，注意，这里的入参并不是指的函数返回的值作为入参，而是说，函数本身作为入参。Lambda 表达式类似于函数式编程，它的形式为 `(x, y) -> x * y`，你可以把它作为一个参数传递到一个方法中。你可能注意到，这个函数仅仅定义了一组计算方法，但是没有提供数据，这是因为我们将数据和策略进行了区分，函数提供的是计算数值的策略，类似于策略模式。

在稍后的例子中，将会看到更多使用函数式编程简化 OOP 设计模式的例子。在一些情况下，它们被称之为语法糖，但不论是从形式还是阅读上，函数式编程都十分优雅，所编写的代码更加简洁。

函数式编程是一种对于 OOP 设计模式的简化，可以由下面这个例子看出：

为了得到一个接口的对象，我们必须手动创建这个接口的一个实例：

```java
Comparator<Integer> comparator = new Comparator<Integer>() {
    public int compare(Integer o1, Integer o2) {return Integer.compare(o1,o2);}
};
TreeSet<Integer> treeSet = new TreeSet<>(comparator);

Comparator<Integer> comparator = (x, y) -> Integer.compare(x,y);
TreeSet<Integer> treeSet = new TreeSet<>(comparator);
```

可以看到，因为 Java 的纯 OO 风格，我们必须为一个方法提供一个类，尽管我们在上面已经使用了匿名内部类简化了创造一个类的步骤（传统过程是创建一个 .class 文件，这个文件包含一个新的类），但是，除了核心的 `Integer.compare(o1,o2)`，其余所有的代码都是冗余的。

而函数式的风格如下所示：我们直接可以创建一个 “函数”，作为一个变量？真的可以吗？当然不可以，这只是看起来像 FP 的 OOP 语法糖，在编译期还是会被解包成为匿名内部类，编译 .class 文件，然后交给 JVM 执行。

尽管如此，这种写法看起来更清晰，优雅，可能唯一的缺点就是，如果你不熟悉 Comparator 接口，你可能不知道这里的 x, y 是什么类型，因为这一切都是 编译器自动推断得来的。

## 面向对象、设计模式和匿名内部类

Lambda 是一种用来促进代码复用的技术，和 OOP 的设计模式一样，或者说，Lambda 表达式能够以 FP 的方式来实现设计模式，而不用传统 OOP 的方式，这种方式间接优雅。

首先，我们要看一看设计模式的使用场景：

首先定义一个 Employee 类，这个类是一个 POJO，包括以下属性：

```java
public class Employee {
    private String name;
    private Integer age;
    private Integer salary; 
    ... 
}
```

然后伪造一个员工类的集合：

```java
List<Employee> employeeList = Arrays.asList(
    new Employee("A",118,2323),new Employee("B",618,23223),new Employee("C",158,2321),
    new Employee("D",118,4543),new Employee("E",128,2466));
```

如果我们现在希望从这一组员工中找年龄大于120岁的，我们要编写一个方法：

```java
public List<Employee> filterEmployees(List<Employee> list) {
    List<Employee> list1 = new ArrayList<>();
    for (Employee e: list) {if (e.getAge() >= 120) {list1.add(e);}}
    return list1;
}
```

然后调用这个方法： `filterEmployees(employeeList)`

这是纯粹的 OOP 设计，但是问题是，我们没有区分可变和不可变的，如果我们需要更改需求，找到年龄大于130岁的员工，那么就不得不重写上述方法，如果我们要过滤年龄的同时需要排序、过滤薪水，那么问题就更加复杂了，为此，我们有了设计模式。设计模式的目的就是为了促进 OOP 的代码复用。

首先我们要面向接口编程，因此要有一个接口 `public interface Filter<T> {boolean need(T t);}`, 这个接口可以为我们提供“策略模式”的策略类所必须的方法 `need`。之后，编写这个接口的不同实现：

```java
class AgeFilter implements Filter<Employee> {
    public boolean need(Employee employee) {return employee.getAge() >= 120;}
}
```

然后重写原来的方法：

```java
public List<Employee> filterEmpInPattern(List<Employee> list, Filter<Employee> filter) {
    List<Employee> list1 = new ArrayList<>();
    for (Employee e: list) {if (filter.need(e)) list1.add(e);}
    return list1;
}
```

这样的话，当我们有了新的需求，就写一个新的策略，使用现在重构过的方法，就可以在相同的数据源上应用不同的策略了。这是“区分变化的和不可变化的”的一个绝佳实例，也就是设计模式中使用最多的“策略模式”。初看起来，这种方式需要的代码行数更多，文件更多，操作更复杂，但是，所有的设计模式都不是为了写 1 + 1 = 2 这种级别的程序服务的，当代码急剧增长，设计模式是唯一能够拯救 OOP 的重要思想和最佳的实践。

以上的设计模式，按照匿名内部类的方式，可以简写成为：

```java
filterEmpInPattern(employeeList, new Filter<Employee>() {
    public boolean need(Employee employee) {return employee.getAge() >= 120;}})
```

这样看上去已经非常的清爽了，但是，里面真正有用的逻辑其实还只是一句话 `employee.getAge() >= 120;`

我们看看 Java 8 Lambda 表达式的写法： `filterEmpInPattern(employeeList, employee -> employee.getAge() >= 120)`。极其的间接和直白。

其实，使用基于 Lambda 的 Stream 流，还可以这样写：

```java
employeeList.stream().filter(e -> e.getAge() >= 120).limit(2).forEach(System.out::println);
```

看起来更棒了。

## Lambda 表达式基础

 `->` 操作符称为箭头操作符/Lambda操作符，比如：`(e) -> { e.getAge() >= 120 }`
 
表达式的左侧是参数列表，参数列表即实现所需的方法参数列表。

表达式的右侧是表达式所需执行的功能，即 Lambda 体。Lambda体即对于接口的实现。
 
因为接口可能无参无返回值、有单个参，无返回值、有多个参，无返回值。以及三种情况有返回值六种情况。

**没有参数，无返回值**

无参无返回值类似于 ` () -> System.out.print("Hello");` 无参不需要提供参数，但是要写小括号。Lambda 体提供了此方法的动作，在这里是打印一个单词。匿名内部类和Lambda内部引用的本地变量本质上还是 final 方法，不过不用加，但是不可 ++。如下所示：

```java
int a = 8;
Runnable runnable = () -> System.out.println("Hello, Lambda in J" + a);
runnable.run();
```

在上面的例子中，runnable 变量是一个 Runnable 类型的对象，只不过我们没有采用 OOP 的方式创建对象，而是采用了 FP 的方式。这类似于 FP “交给子类”设计模式的写法。因为我们得到的是一个 Runnable 类型的对象，自然可以调用相应的方法。

注意，这里的 a 变量虽然不用写 final（J8），但是在 Lambda 体重默认含义依旧是不可能更改的 final 类型。如果在 Lambda 体中写 a++ 就会报错。

**有一个参数，无返回值**

类似 `(e) -> {System.out.print(e)};`，有一个参数，则在小括号写一个参数的引用，注意：

- 参数列表可以提供类型信息，也可以不提供
- 当参数列表只有一个参数，可以省略小括号。
- 当实现只有一条语句，可以省略花括号。

```java
Consumer<Employee> consumer = employee -> System.out.println(employee);
consumer.accept(new Employee("Marvin",null,null));
```

**有多个参数，无返回值**

对于有多个参数，无返回值： 

- 如果有多条语句，使用 {}，如果只有一条语句，可以省略 {}。

`(a, b) -> {System.out.print(a + b)};`

`(a, b) -> System.out.print(a + b)`


**有零个/单个/多个参数，有返回值**

参数列表需要注意：

- 对于有一个参数，有返回值，参数小括号可省略

Lambda 体需要注意：

- 如果结构体有多条语句，则使用 {}。
- 如果结构体只有一条语句，则 return 可以不写。

下列语句等同：

`(a, b) -> { if (a > b) return a; else return b; }` 

`(c, d) -> ((c > d) ? c : d);`

## Lambda 表达式的使用

**Lambda 表达式语法小结**

左边为参数列表，类型自动推断，可以不写类型。如果没有参数，使用 （），如果有一个参数，则可以省略（）。如果有多个参数，则不可省略（）。

右边是 Lambda 体，对于单条语句，可以不写 {}，对于多条语句，不可不写 {}。对于有返回值的单条语句，可以省略 return 和 {}，对于有返回值的多条语句，均不可省略。

概括为：左右遇一括号省，左侧推断类型省。能省则省。

**Lambda 表达式的函数式接口声明**

Lambda 表达式需要函数式接口（接口中只有一个抽象方法）的支持。可以使用 @FunctionalInterface 修饰接口，编译器会检查此接口是否为函数式接口。

```java
@FunctionalInterface
public interface Filter<T> {  boolean need(T t);  }
```

**Lambda 的两种不同使用方法**

1. 抽像工厂 设计模式的实现

声明一个函数式接口，然后提供一个策略方法。一般不是这样直接声明一个“匿名抽象类”，然后调用其方法。这种用法可以看作是交给子类的设计模式的实现，一般我们使用组合而不是继承：

```java
MyFunction function = d -> d * 1000;
System.out.println(function.operation(100));
```

2. 策略模式 设计模式的实现

正宗的用法是为函数式接口提供一个能够接受策略的方法/构造器，然后调用此构造器返回值。这种用法可以看作是策略模式的FP实现，最为常用。 此外，这个看起来更像 FP。在方法中提供稳定的代码，而将更为容易变化的部分交给 Lambda 表达式来完成，促进变化和不变的分离。

```java
public Integer doMath(Integer integer, MyFunction function) {
    return function.operation(integer);
}
System.out.println(doMath(100, d -> d * 1000));
```

在真实情况下，常见的用法为 FP 作为策略（函数作为参数），如下代码按照年龄比较，如果年龄相同，则比较姓名，使用了 Lambda 表达式。

```java
Collections.sort(employeeList, (e1, e2) -> {
    if (e1.getAge() == e2.getAge())
        return e1.getName().compareTo(e2.getName());
    else
        return Integer.compare(e1.getAge(), e2.getAge());
});
```

## Lambda 函数式接口支持

Java 8 在 util.Function 类提供了几个函数式接口的支持，这样的话，在使用 Lambda 表达式的时候，就可以直接编写方法接受这些接口类型，以提供方法对于 Lambda 表达式的支持。换句话说，这些接口可以看作是“策略模式”的模板。

常见的接口因为入参的类型和个数、返回值的类型和个数不同而不同，常见的内置函数式接口为：

 * Consumer<T> void accept(T t) 消费型接口 用于消费
 * Supplier<T> T get() 供给型接口 用于单纯供给
 * Function<T,R> R apply(T t) 函数型接口 类型转换
 * Predicate<T> boolean test(T t) 断言型接口 用于判断
    
**Consumer 接口**

消费者接口只进不出，接受一个类型参数，不返回信息，因此称之为消费。常见的用法为：`happy(1000, m -> System.out.println("Out of " + m ));` 单纯用于消费。常用来进行日志记录，审计等等。

```java
public void happy(Integer money, Consumer<Integer> consumer) {consumer.accept(money);}
```

**Supplier 接口**

供给接口不接受参数，只返回类型信息，因此称之为供给。

Supplier 通常用来接收配置信息/运算方法，比如从某地加载一个配置文件，决定列表数字的产生方法等。

注意，这些配置信息并非直接可以得到，必须经过运算（比如从易变的地方加载，获取等）。这些运算属于“容易变化”的部分，所以使用策略模式和 FP 来保持解，FP 用于获得这些易于变化的信息，作为策略模式整合到传统方法中。

```java
public List<Integer> getRandomList(Integer count, Supplier<Integer> func) {
    Random random = new Random();
    List<Integer> result = new ArrayList<>();
    for (int i= 0; i < count; i++) {
        result.add(func.get());
    }
    return result;
}
//usage
getRandomList(20,() -> new Random().nextInt(50)).forEach(System.out::println);
```

**Function 接口**

Function<T,R> 常常用来对数据进行修剪，比如字符串处理（基于 Lambda 的 Stream 可以更好、更快的处理）。下面的例子为字符串转变大写、去除多余字符、进行字符串解析，这些操作过于琐碎，因此不适合放在传统方法中（写起来麻烦），因此由 Lambda 表达式作为参数直接传递。

```java
public String strHandler(String stuff, Function<String,String> func) { return func.apply(stuff);}
System.out.println(strHandler("hello",(s -> s.toUpperCase())));
System.out.println(strHandler("\t\t\thello",(s -> s.trim())));
System.out.println(strHandler("20:15:15 23211",(s ->
    s.split(":")[0] + s.split(":")[1] +
    s.split(":")[2].split(" ")[0] + s.split(":")[2].split(" ")[1]
)));
```

**Predicate 接口**

断言型 Lambda 用于对于传入的数据进行判断，返回 bool 值。比如下面这个方法，接受一个判断，然后如果通过判断，则加入列表，反之，不加入。这里的判断是，如果首字母是大写，则通过判断。

```java
public List<String> strListFilter(List<String> stuff, Predicate<String> isGood) {
    List<String> result = new ArrayList<>();
    stuff.forEach(s -> { if (isGood.test(s)) result.add(s); } );
    return result;
}
//usage
strListFilter(Arrays.asList("H","B","o"), 
              s -> Character.isUpperCase(s.toCharArray()[0])).forEach(System.out::println);
```

**四大核心接口的子接口**

除此之外，还有这些核心接口的子接口：

BiFunction 接受两个参数，返回第三个参数类型信息。

BiConsumer，接受两个参数，不返回。

BinaryOperator 接受两个相同参数类型参数，返回此类型的一个参数。

BiPredicate 方法接受两个两个类型参数，返回bool值

ToInt/Long/DoubleFunction 接受一个类型的一个参数，返回一个int，long，double类型数据。

Int/Long/DoubleFunction 接受int，long，double类型的一个参数，返回一个类型的参数。

**注意** 使用这些接口的时候，尤其需要注意，基于策略模式的 FP，需要区分变化和不变，将变化的封装在 FP 中，将不变的放置在方法中，然后方法去接口 FP 作为参数。实际情况，我们的策略可能是一个判断（使用断言接口），或者是一个转换（使用函数接口），或者是一个提供（使用供给接口），或者是一个消费（使用消费接口），或者需要好几个参数（使用Bi+四大接口）。

# Method Ref

Lambda 表达式是 Java 8 一系列新特性的基础，Method Ref 方法引用就是建立在 Lambda 表达式基础上的。使用方法引用，能够编写出更加简洁的 Lambda 表达式（看起来完全不像Java）

## 从匿名内部类到方法引用
    
对于匿名函数，如果函数体已经有了现有的实现，那么可以继续简写为方法引用的形式。

```java
//基本的 Lambda 表达式
Consumer<String> consumer = s -> System.out.println(s);
consumer.accept("Hello, World");

//实现的返回值 void，参数列表 Object，和接口的要求均匹配，可以将匿名函数改写成方法引用
Consumer<String> simpConsumer = System.out::println;
simpConsumer.accept("Hello, Again!");
```

具体而言，可以省略匿名函数的参数列表和匿名函数函数符号，只保留右边的实现。但是，此时的实现应该改成 `现有的对象::对象的方法实现` 这种形式。

换句话说，此方法实现提供了编译器所需的匿名函数参数列表信息和实现信息。此方法实现的参数列表必须和匿名函数省略的参数列表匹配，返回值匹配。因为此处省略了方法列表，因此不要写（），空括号默认含义是空参数，如果不写则是自动推断。


常见的语法格式有：

- 对象::方法名（此处方法名不加括号，省略参数信息，下同）

  `Employee e = new Employee(); e::getName 等同于 (e) -> e.getName();`
  
  
- 类::静态方法名

  `Math::random 等同于 () -> Math.random()`
  
  
- 类::对象方法名

  `String::equals 等同于 (x,y) -> x.equals(y)`
  `Employee::getName 等同于 (e) -> e.getName()`
  
  
- 类::new(特殊,又称构造器引用)

  `ClassName::new 等同于 (a) -> new ClassName(a)`
  
  
- 数组类型::new(特殊，又称数组引用)

  `String[]::new 等同于 (l) -> new String[l];`
  

## 方法引用的语法概述

**对象::方法名**

注意，Lambda 表达式本身就隐含了接口的参数类型信息和方法签名信息，方法引用在 Lambda 表达式具有默认实现的情况下，在其基础上省略了参数要写两次的写法，直接从此实现的入参推断 Lambda 表达式传入的参数个数和类型信息。因此，实现的入参和返回值必须与 Lambda 所使用的接口保持一致。

```java
Employee employee = new Employee("Marvin",null,null);
Supplier<String> supplier = employee::getName;
//此处提供的是 name 属性, 如果是 getAge 则参数类型不匹配
System.out.println(supplier.get());
```

**类::静态方法**

```java
Comparator<Integer> comparator_bad = (x, y) -> Integer.compare(x,y);
System.out.println(comparator_bad.compare(10,20));

Comparator<Integer> comparator = Integer::compareTo;
System.out.println(comparator.compare(10,20));
```

**类::方法名**

注意，这种情况下，和 对象::方法名不同，我们是调用入参第一个参数充当此类，然后调用其方法名完成对于第一个入参的调用，这个方法可以传递参数，传递的参数匹配入参的第二个到最后一个参数。

换句话说，这种语法充分利用了入参，入参第一个参数被当作调用对象，之后所有参数被当作调用对象的方法的入参。

```java
 BiPredicate<String,String> biPredicate = (x, y) -> x.equals(y);
System.out.println(biPredicate.test("a","b"));

BiPredicate<String,String> biPredicate_new = String::equals;
```

**类::new**

这种语法可以构造类的构造器，返回构造对象，构造器的入参由Lambda 实现的接口所需要的入参决定，不一定必须是无参构造器，这取决于接口，比如 Supplier 接口没有参数，通过此接口的 FP，自然是无参构造器，而 Function 接口有两个参数，因此，会去调用泛型中的类型的两个参数的构造器进行构造并且返回。

这种方法适合于 “策略模式”或者“交给子类”设计模式的简写，都非常方便。尤其是构造一个类对象，这种方法相比 new 关键字看起来更简洁。

```java
Supplier<Employee> employeeSupplier = () -> new Employee();
Supplier<Employee> employeeSupplier_new = Employee::new;
//无参构造器，无参的原因是，Supplier没有参数，因此参数列表为空

Function<Integer, Employee> employeeFunction = (x) -> new Employee(x);
Function<Integer, Employee> employeeFunction_new = Employee::new;
//一个参数构造器，一个参数原因是，接口需要一个参数 INTERGETER 类型，因此找到构造器有一个相对应参数类型的构造器。
```

**数组类型::new**

这种语法可以创建一个数组引用，`x -> new Type[x]` 变成了 `Type[]::new`，传入的数组长度被引用的泛型所限定，看起来更简洁。数组方法引用相比较直接创建数组，能够充分体现出 FP 相比较 OOP 的特点，提供方法而不是一个具体的实现。

```java
Function<Integer, String[]> function = x -> new String[x];
System.out.println(function.apply(10).length);

Function<Integer, String[]> function_new = String[]::new;
System.out.println(function_new.apply(20).length);
```

# Stream API

## 创建 Stream 流

Stream API 是一种快速处理 Java 容器类元素的一种类似于 IO 流的数据流。创建 Stream API 有以下几种方式，分别是通过 Collection 各个接口和实现类的 `.stream/.porallelStream` 获得，通过 Arrays.stream 方法获得，通过 Stream.xxx 获得。通过 Stream 类型获得的流又分为 of 构造普通流，iterate 迭代流，generate 生成流等不同类型的一般或者纯方法型的流。

```java
//1. 通过 Collection 的 stream 或者 porallelStream 获得串行或者并行流
List<String> strings = new ArrayList<>();
Stream<String> stringStream = strings.stream();
//2. 通过 Arrays 的 stream 方法获得
Stream<String> stringStream2 = Arrays.stream(new String[10]);
//3. 通过 Stream 类型获得
//普通流
Stream<String> stringStream3 = Stream.of("hello","world");
//迭代无限流：
Stream<Integer> stream = Stream.iterate(0, integer -> integer + 20);
stream.limit(10).forEach(System.out::println);
//生成无限流：
Stream<Double> stream2 = Stream.generate(() -> Math.random());
stream2.limit(10).forEach(System.out::println);
```

普通流很简单，无限流需要接受一个 Lambda 表达式，以提供满足此方法/策略/特征的流。

在开始之前，为之前的 Employee 类提供 Status 的枚举类型，一个 get/set 方法，创建伪造列表以创建流。

```java
private Status status;
public enum Status {
    FREE, BUSY, VOCATION;
}
List<Employee> employeeList = Arrays.asList(
        new Employee("A",118,2323, Employee.Status.FREE),
        new Employee("B",618,23223, Employee.Status.BUSY),
        new Employee("C",158,2321, Employee.Status.VOCATION),
        new Employee("D",118,4543, Employee.Status.BUSY),
        new Employee("E",128,2466, Employee.Status.VOCATION),
        new Employee("F",198,2466, Employee.Status.FREE)
);
```



## 中间操作

Stream 有三种类型的操作，分别是 获取流、流的中间操作、终止操作。在上一部分介绍了获取流的操作，这里主要介绍流的中间操作。

**Filter 过滤流**

Filter 方法会截获经过流的元素，然后对其进行判断，如果满足，则继续流程，否则终止此（元素）流程，然后开始下一元素流程。

```java
employeeList.stream()
    .filter( e -> {
        System.out.println("Middle operation " + e);
        return e.getAge() > 130;
    })
    .forEach(System.out::println);
```

Filter 传入的是一个 Function 接口的类，也就是一个 Function 的 Lambda 实现，其提供了对于迭代的每个元素进行的判断。使用 forEach 终止操作，进行迭代（这是一个终止操作）。

需要注意，流的中间操作没有结果，必须有“终止操作”，比如 forEach。多个中间操作可以连接起来形成流水线。除非有终止操作，则不会执行。有终止操作后，才去执行全部内容，这称之为**惰性求值**。

**Limit 截取流**

Limit 方法会截断流，使经过其的元素不超过某一数量，如果超过，则终止之后所有元素的流程。

需要注意，结合使用 limit(2) 和 filter 的时候，当 filter 得到两个元素后，其迭代过程就自动被截断了。这个过程称之为**短路**，提高了运行效率（实际上就是一个大的循环，每次遍历判断所有中间操作，只要满足所有中间操作条件，就不再继续执行，直接到终止操作。而不是对每个中间操作执行一次循环）。

```java
employeeList.stream().limit(3).forEach(System.out::println);

employeeList.stream()
            .filter( e -> {
                System.out.println("短路中...");
                return e.getAge() > 130;})
            .limit(2)
            .forEach(System.out::println);
```

**Skip 跳过元素**

```java
employeeList.stream().skip(3).forEach(System.out::println);
```

**Distinct 元素去重**

```java
employeeList.stream().distinct().forEach(System.out::println);
```

distinct 方法用于去重（根据 hashCode 和 equals 去重），需要提前实现自定义类中的这两个方法。

**Map 应用函数操作**

·